In [2]:
import pandas as pd
import numpy as np

## Label-studio 파일 읽어오기

In [16]:
file_name = 'label-studio.csv'
DF = pd.read_csv(file_name)

## TagTog 형식 맞추기

In [18]:
new_df = pd.DataFrame(columns=["sentence","sentence_with_entity","subject_entity","object_entity","class"])

In [19]:
new_df['sentence'] = DF['text']

In [20]:
DF['label'] = DF['label'].str.slice(1,-1)

In [22]:
DF['label'] = DF['label'].map(lambda x : x.replace('labels','type'))

In [135]:
def return_sbj(data):
  result = [{'start':0,'end':0,'text':'','type':''},{'start':0,'end':0,'text':'','type':''}]
  sp = data.split(',')

  result[0]['start'] = int(sp[0].split(':')[1])
  result[0]['end'] = int(sp[1].split(':')[1]) - 1
  result[0]['text'] = sp[2].split(':')[1].strip()
  tps = sp[3].split('-')
  result[0]['type'] = tps[1].lower()


  if 'SBJ' in tps[0]:
    sbj_idx = 0
  elif 'OBJ' in tps[0]:
    sbj_idx = 1

  result[1]['start'] = int(sp[4].split(':')[1])
  result[1]['end'] = int(sp[5].split(':')[1]) - 1
  result[1]['text'] = sp[6].split(':')[1].strip()
  result[1]['type'] = sp[7].split('-')[1].lower()

  if sbj_idx == 0:
    return result[0]
  elif sbj_idx == 1:
    return result[1]
  else:
    print('err')

In [136]:
def return_obj(data):
  result = [{'start':0,'end':0,'text':'','type':''},{'start':0,'end':0,'text':'','type':''}]
  sp = data.split(',')

  result[0]['start'] = int(sp[0].split(':')[1])
  result[0]['end'] = int(sp[1].split(':')[1]) - 1
  result[0]['text'] = sp[2].split(':')[1].strip()
  tps = sp[3].split('-')
  result[0]['type'] = tps[1].lower()


  if 'SBJ' in tps[0]:
    sbj_idx = 0
  elif 'OBJ' in tps[0]:
    sbj_idx = 1

  result[1]['start'] = int(sp[4].split(':')[1])
  result[1]['end'] = int(sp[5].split(':')[1]) - 1
  result[1]['text'] = sp[6].split(':')[1].strip()
  result[1]['type'] = sp[7].split('-')[1].lower()

  if sbj_idx == 0:
    return result[1]
  elif sbj_idx == 1:
    return result[0]
  else:
    print('err')

In [137]:
def return_class(data):
  result = [{'start':0,'end':0,'text':'','type':''},{'start':0,'end':0,'text':'','type':''}]
  sp = data.split(',')

  tps = sp[3].split('-')
  tp1 = tps[1].lower()
  cls1 = tps[2].split('"')[0].lower()


  if 'SBJ' in tps[0]:
    sbj_idx = 0
  elif 'OBJ' in tps[0]:
    sbj_idx = 1

  tps = sp[7].split('-')
  tp2 = tps[1].lower()
  cls2 = tps[2].split('"')[0].lower()

  if cls1=='no_relation' and cls1==cls2:
    return 'no_relation'
  elif sbj_idx == 0 and cls1==cls2:
    return f'{tp1}:{cls1}'
  elif sbj_idx == 1 and cls1==cls2:
    return f'{tp2}:{cls1}'
  else:
    print('err')

In [138]:
new_df['subject_entity'] = DF['label'].apply(lambda x : return_sbj(x))

In [139]:
new_df['object_entity'] = DF['label'].apply(lambda x : return_obj(x))

In [140]:
new_df['class'] = DF['label'].apply(lambda x : return_class(x))

In [141]:
def sen_w_entity(sen,sbj,obj):
  result = ''
  def cleaning(word):
    return word.replace('"','').replace("'",'')
  if sbj['start'] < obj['start']:
    result = f"{sen[:sbj['start']]}<sbj:{cleaning(sbj['text'])}>{sen[sbj['end']+1:obj['start']]}<obj:{cleaning(obj['text'])}>{sen[obj['end']+1:]}"
  else:
    result = f"{sen[:obj['start']]}<obj:{cleaning(obj['text'])}>{sen[obj['end']+1:sbj['start']]}<sbj:{cleaning(sbj['text'])}>{sen[sbj['end']+1:]}"
  return result

In [142]:
new_df['sentence_with_entity'] = new_df.apply(lambda x : sen_w_entity(x['sentence'],x['subject_entity'],x['object_entity']),axis=1)

## 기존 구글 스프레드 시트에 붙이기

In [144]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json
import glob
import re
import os
import pandas as pd
import html

In [104]:
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'api_key.json', scope)
gc = gspread.authorize(credentials)

# 자신이 만든 구글 시트 이름
sh = gc.open('반려동물_데이터셋생성')
sheet = sh.worksheet('시트1')

In [145]:
# 구글시트에 입력
values = sheet.get_all_values()

header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)

column_list = ["sentence","sentence_with_entity","subject_entity","object_entity","class"]
data = data[column_list]

sen_list =  list(data.sentence_with_entity.values)
print(f'sentence list : {len(sen_list)}')

sheet.resize(len(sen_list)+len(new_df)+1,10)
start_idx = 473
list_range = f"a{start_idx}:e{start_idx+len(new_df)}"
cell_list = sheet.range(list_range)

col_num = len(column_list)
print(f'c : {len(cell_list)}, {col_num}')
for i in range(len(new_df)):
    cell_list[(col_num*i)].value = new_df['sentence'][i]
    cell_list[(col_num*i)+1].value = new_df['sentence_with_entity'][i]
    cell_list[(col_num*i)+2].value = str(new_df['subject_entity'][i])
    cell_list[(col_num*i)+3].value = str(new_df['object_entity'][i])
    cell_list[(col_num*i)+4].value = new_df['class'][i]
sheet.update_cells(cell_list)

sentence list : 494
c : 120, 5


{'spreadsheetId': '1dcLAlfOTb_n7NSUN3y9MTzy7_MBvrSsFoDXyR1F-AaE',
 'updatedCells': 120,
 'updatedColumns': 5,
 'updatedRange': "'시트1'!A473:E496",
 'updatedRows': 24}